# Requirements

In [ ]:
!pip install datasets transformers
!pip install git+https://github.com/LIAAD/yake
!pip install hazm

In [ ]:
import collections
import helper
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
cc = SmoothingFunction()

import tensorflow as tf
from transformers import AutoModelForMaskedLM, AutoTokenizer

from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pickle
import yake
from itertools import chain
import nltk
import pickle
from statistics import mean
import math
import pandas as pd
from itertools import product
import sys
import ast
import string
import unicodedata
import random
import re
import numpy as np
import os
import io
import time
import pandas as pd
import hazm
import gc
from termcolor import colored
from itertools import chain
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)

In [ ]:


from google.colab import drive
drive.mount('/content/drive')


all_data_address = '.../ProsPoemParallelDataset_augmented.pickle'
indexed_vocabs_address = '.../indexed_vocabs.pickle'
freqs_path = '.../freqs.pickle'
adjacency_graph_adress = '.../AdjacencyPickle.pickle'
translator_output_address = '.../MultiHeadAttention_Poetry_17_cleaned_data.csv'
poetry_beam_addreass = '.../PoetryBeamN.pickle'
output_address = '.../Results/final_output_BeitBert.csv'
G_ghafie_address = '.../RhymeGraph.pickle'
motaradef_address = '.../motaradef.pickle'
rhymes_address = '.../rhymes.pickle'
unw_tokens_address = '.../unw_wokens.pickle'


In [ ]:

poetryBeam = pd.read_pickle(poetry_beam_addreass)
rhymes = pd.read_pickle(rhymes_address)
all_unw_tokens = pd.read_pickle(unw_tokens_address)
G_ghafie = pd.read_pickle(G_ghafie_address)
freqs = pd.read_pickle(freqs_path)
G = pd.read_pickle(adjacency_graph_adress)
indexed_vocabs = pd.read_pickle(indexed_vocabs_address)
motaradef = pd.read_csv(motaradef_address)

In [ ]:
motaradef.loc[:, 'synonym'] = motaradef.loc[:, 'synonym'].apply(lambda x: ast.literal_eval(x))

In [ ]:
pd.set_option('display.max_rows', 100)
gc.collect()

# Phase || Huristic 

In [ ]:
# GUIDE

# R (Robaei - Ghazal - Ghaside) - D (Dobeiti) - GH (Ghete) - M (Masnavi)

# R: 
# O-----------    O-----------    
# O-----------     -----------    

# D: 
# O-----------    O-----------    
# O-----------    O-----------    

# GH: 
# O-----------     -----------    
# O-----------     -----------    

# M: 
# O-----------    O-----------    
# X-----------    X-----------    

In [ ]:
all_fdist = dict()
for key in list(indexed_vocabs.keys()):
  fdist = nltk.FreqDist(indexed_vocabs[key])
  all_fdist[key] = fdist
print(len(all_fdist))

most_common = []



top_n = 5
for key in list(all_fdist.keys()):
    most_common += [each[0] for each in all_fdist[key].most_common()[:top_n]]
most_common = list(set(most_common))
print(most_common, len(most_common))

In [ ]:
# Mitra has modified this section-------------------------------------

language = "fa"
max_ngram_size = 1
deduplication_thresold = 0.99
deduplication_algo = 'seqm'
windowSize = 1
features = None

def extract_keys(corpus, numOfKeywords):
    kw_extractor = yake.KeywordExtractor(lan=language, 
                                         n=max_ngram_size,
                                         dedupLim=deduplication_thresold, 
                                         dedupFunc=deduplication_algo, 
                                         windowsSize=windowSize, 
                                         top=numOfKeywords, 
                                         features=features)
    
    keywords = kw_extractor.extract_keywords(corpus)
    # the lower the score the more relevant the key
    keys = [key[0] for key in sorted(keywords, key =  lambda x: x[1])]
    return keys
def extract_single_keys(mapped, numOfKeywords):
  '''
  input sample: 
  mapped = [
          ['کار', 'عارف', 'راست', 'کو', 'نه', 'احول', 'است', '،', 'چشم', 'او', 'از', 'کشت\u200cهای', 'اول', 'است', '.'],
          ['ای', 'بسا', 'کفار', 'را', 'سودای', 'دین', '،', 'بند', 'او', 'ناموس', 'و', 'کبر', 'و', 'این', 'کار', '.']
  ]

  output sample: [['کار', 'چشم', 'عارف'], ['دین', 'بند', 'کار']]
  '''
  single_keys = []
  for each in mapped:
    keys = extract_keys(' '.join(each), numOfKeywords)
    single_keys.append(keys)
  return single_keys


def extract_joint_keys(mapped, numOfKeywords):
  '''
  input sample: 
  mapped = [
          ['کار', 'عارف', 'راست', 'کو', 'نه', 'احول', 'است', '،', 'چشم', 'او', 'از', 'کشت\u200cهای', 'اول', 'است', '.'],
          ['ای', 'بسا', 'کفار', 'را', 'سودای', 'دین', '،', 'بند', 'او', 'ناموس', 'و', 'کبر', 'و', 'این', 'کار', '.']
  ]

  output sample: [['چشم', 'دین', 'بند']]
  '''
  joint_keys = []
  mapped_joined = ' '.join([' '.join(each) for each in mapped])
  keys = extract_keys(mapped_joined, numOfKeywords)
  joint_keys.append(keys)

  return joint_keys


def extract_all_keys(single_keys, joint_keys, numOfKeywords):
  '''
  input sample: 
  [['کار', 'چشم', 'عارف'], ['دین', 'بند', 'کار']]   [['چشم', 'دین', 'بند']]

  output sample: {'بند', 'دین', 'کار', 'چشم', 'عارف'}
  '''
  final_keys = []
  for each1 in single_keys + joint_keys:
    for each2 in each1:
      final_keys.append(each2)
  final_keys = set(final_keys)
  return final_keys

def calculate_fks(single_keys, joint_keys, final_keys):
  '''
  input sample: {'بند', 'دین', 'کار', 'چشم', 'عارف'}

  output sample: [2, 2, 2, 2, 1]
  '''
  single_joint = []
  for each in single_keys + joint_keys:
    for each2 in each:
      single_joint.append(each2)
  fks_tmp = {}
  for each in single_joint:
    if each in fks_tmp:
      fks_tmp[each] = fks_tmp[each] + 1
    else:
      fks_tmp[each] = 1
  fks = [fks_tmp[each] for each in final_keys]
  return fks
# find_synonyms is used to extract synonyms for a list of keys.
# Our objective here is to extract n synonyms for each key (where n is equal to fks[key])
# provided that each key's length is less than 10, is only a single word, and has a frequency of bigger than 0.
def find_synonyms(keys, fks):

    synonyms = {}
    for each in keys:
        synonyms[each] = []
    for i, each in enumerate(keys):
        if each in motaradef['word'].to_list():

            tmp = motaradef[motaradef['word'] == each]['synonym'].tolist()[0]


            #tmp_mtr = [each2[1:-1].strip() for each2 in tmp[0].split(', ')]
            # tmp_mtz = [each[1:-1].strip() for each in tmp[1].split(', ')]   
            #print()
            #print(tmp)

            for each2 in tmp:
                #print(each2)
                this_freq = sum([freqs[str(i + 1)][each2] for i in range(10)])
                if len(each2) < 10 and len(each2.split()) == 1 and this_freq > 0:
                    synonyms[each].append(each2)
                    if fks[i] == len(synonyms[each]):
                        break
    return synonyms


# Similar to the above function, but only a single synonym is returned.
def find_one_synonym(all_keys, key):

    #print()
    #print('finding one syn ', )
    #print(motaradef[motaradef['word'] == key]['synonym'].tolist())

    #try:
    if key in motaradef.word.tolist():

        tmp = motaradef[motaradef['word'] == key]['synonym'].tolist()[0]
    
        for each2 in tmp:
            #print(each2)
            this_freq = sum([freqs[str(i + 1)][each2] for i in range(10)])
            if len(each2) < 10 and len(each2.split()) == 1 and each2 not in all_keys and this_freq > 0:
                return each2
    return key


# find_keys_synonyms combines all keywords and synonyms
def find_keys_synonyms(keys, synonyms):
    return keys + [each2 for each in list(synonyms.values()) for each2 in each]


# This function calculates a keyword/synonym 's position
# Try Except is used to handle exceptions in case a keyword/synonym is OOV
# find_key_poses returns both positions and the updated keys_synonyms list (which has no OOV word in it)
def find_key_poses(keys_synonyms):
  key_poses = {}
  for each in keys_synonyms:
    try:
      key_poses[each] = calculate_position(each, list(key_poses.values()))
    except:
      key_poses[each] = -1
  key_poses_main = {}
  for each in list(key_poses.keys()):
    if key_poses[each] != -1:
      key_poses_main[each] = key_poses[each]
  return key_poses_main, list(key_poses_main.keys())


# find what keywords have been suggested per index
def find_indices_suggestions(key_poses):
  indices_suggestions = {}
  for i in range(10):
    indices_suggestions[str(i + 1)] = [each for each in list(key_poses.keys()) if key_poses[each] == i + 1]
  return indices_suggestions

# In order to normalize indices_suggestions 
# we either remove or add a keyword



# d is a parameter which measures the 
# total number of differences between keywords' indices
# E.g. => input: HELLO NONE MITRA NONE HOW ARE NONE YOU -> 
# HELLO - MITRA = 2
# MITRA - HOW = 2
# HOW - ARE = 1
# ARE - YOU = 2
# d = 7
def calculate_d(inp, mask_token):
  d = 0
  tmp_idx = None
  for i in range(len(inp)):
    if inp[i] != mask_token:
      if tmp_idx == None:
        tmp_idx = i
      else:
        d += i - tmp_idx
        tmp_idx = i
  return d


# g is a parameter which measures how likely are 
# two words to be adjacent in a mesra
# E.g. => input: MITRA NONE HOW ARE NONE YOU -> 
# MITRA - HOW = 0.65
# MITRA - ARE = 0.49
# MITRA - YOU = 0.76
# HOW - ARE = 0.4
# HOW - YOU = 0.9
# ARE - YOU = 0.3
# g = mean([0.65, 0.49, 0.76, 0.4, 0.9, 0.3])
def check_duplicate(track_list, idx1, idx2):
  if idx1 == idx2:
    return False
  if str(idx1) + '_' + str(idx2) in track_list or str(idx2) + '_' + str(idx1) in track_list:
    return False
  return True


# This function checks if any of the two input words is among the top_n percent of most frequent words
def check_if_most_common(inp1, inp2):
    if inp1 in most_common or inp2 in most_common:
        return True
    return False


def calculate_g(words, mask_token):
  weights = []
  words = [each for each in words if each != mask_token]
  track_list = []
  for i in range(len(words)):
    for j in range(len(words)):
      if check_duplicate(track_list, i, j):
        track_list.append(str(i) + '_' + str(j))
        tmp_i, tmp_j = i + 1, j + 1
        if G.has_edge(words[i], words[j]) and not check_if_most_common(words[i], words[j]):
          weights.append(G[words[i]][words[j]]['weight'])
  if len(weights) > 0:
    return mean(weights)
  else:
    return 0

def sigmoid(x):
  return 1 / (1 + math.exp(-x))


In [ ]:
# calculate n_r to quantify how many keywords (in total) are before (right-side) a specific index
def calculate_n_r(idx, previous_positions): 
    # idx (1 - 10)
    counter = 0
    for each in previous_positions:
        if each < idx:
            counter += 1
    return counter + 1


# calculate n_l to quantify how many keywords (in total) are after (left-side) a specific index
def calculate_n_l(idx, previous_positions): 
    # idx (1 - 10)
    counter = 0
    for each in previous_positions:
        if each > idx:
            counter += 1
    return counter + 1


# frn (first right neighbour) - fln (first left neighbour)
# calculate frn_d to measure the distance between the index to its first right-side neighbour
def calculate_frn_d(idx, previous_positions): 
    # idx (1 - 10)
    counter = 0
    tmp_idx = idx - 1
    while tmp_idx != 0:
        counter += 1
        if tmp_idx in previous_positions:
            break
        tmp_idx -= 1
    return counter + 1, tmp_idx


# calculate fln_d to measure the distance between the index to its first left-side neighbour
def calculate_fln_d(idx, previous_positions): 
    # idx (1 - 10)
    counter = 0
    tmp_idx = idx + 1
    while tmp_idx != 11:
        counter += 1
        if tmp_idx in previous_positions:
            break
        tmp_idx += 1
    return counter + 1, tmp_idx


# calculate frn_agg to quantify the aggregation of frn
def calculate_frn_agg(frn_idx, previous_positions): 
    return previous_positions.count(frn_idx) + 1


# calculate fln_agg to quantify the aggregation of fln
def calculate_fln_agg(fln_idx, previous_positions): 
    return previous_positions.count(fln_idx) + 1


# calculate rns (right neighbour score)
def calculate_rns(n_r, frn_d, frn_agg):
    return (1 / n_r) * (frn_d / frn_agg)

    
# calculate lns (left neighbour score)
def calculate_lns(n_l, fln_d, fln_agg):
    return (1 / n_l) * (fln_d / fln_agg)


# calculate the aggregation at the candidate index
def calculate_idx_agg(idx, previous_positions): 
    return previous_positions.count(idx) + 1


# Given a word, we sort the indices among which the word has been seen the most.
def calculate_position(each_key, previous_positions):
    one_to_ten = []
    for i in range(len(freqs.keys())):
        this_freqs = freqs[str(i + 1)][each_key]
        one_to_ten.append(this_freqs)
    one_to_ten_scored = []

    for i, idx_freq in enumerate(one_to_ten):

        idx = i + 1
        n_r = calculate_n_r(idx, previous_positions)
        frn_d, frn_d_idx = calculate_frn_d(idx, previous_positions)
        frn_agg = calculate_frn_agg(frn_d_idx, previous_positions)
        n_l = calculate_n_l(idx, previous_positions)
        fln_d, fln_d_idx = calculate_fln_d(idx, previous_positions)
        fln_agg = calculate_fln_agg(fln_d_idx, previous_positions)
        rns = calculate_rns(n_r, frn_d, frn_agg)
        lns = calculate_lns(n_l, fln_d, fln_agg)
        idx_agg = calculate_idx_agg(idx, previous_positions)
        the_score = 0
        if idx_freq != 0:
            the_score = (abs(rns - lns) / idx_agg) * math.log(idx_freq, 10)
        one_to_ten_scored.append([idx, the_score])
    one_to_ten_scored_sorted = sorted(one_to_ten_scored, key =  lambda x: x[1], reverse=True)
    return one_to_ten_scored_sorted[0][0]
    # if len(one_to_ten) < 2:
    #     max_value = max(one_to_ten)
    #     max_index = one_to_ten.index(max_value) 
    # else:
    #     max_value = one_to_ten_sorted[random.randint(0, 1)]
    #     max_index = one_to_ten.index(max_value) 
    # return max_index + 1


def clean_poetry(inp):
  poetry = inp.split()[1:-1]
  poetry = poetry[:poetry.index('<sep>')] + ['،'] + poetry[poetry.index('<sep>') + 1:]
  return poetry



# In order to normalize indices_suggestions 
# we either remove or add a keyword

# This def removes a keyword from
# keys_synonyms and updates key_poses and 
# indices_suggestions
def remove_keyword(inp, keys_synonyms):
    
  keys_synonyms.remove(inp)
  key_poses, keys_synonyms = find_key_poses(keys_synonyms)
  indices_suggestions = find_indices_suggestions(key_poses)
  #print(inp, 'DELETED')
  return keys_synonyms, key_poses, indices_suggestions

# This def adds a keyword to keys_synonyms 
# and updates key_poses and indices_suggestions
def add_keyword(inp, keys_synonyms):
    
  the_synonym = find_one_synonym(keys_synonyms, inp)
  keys_synonyms.append(the_synonym)
  key_poses, keys_synonyms = find_key_poses(keys_synonyms)
  indices_suggestions = find_indices_suggestions(key_poses)
  #print(the_synonym, 'ADDED')
  return keys_synonyms, key_poses, indices_suggestions

In [ ]:

def rnd_rhyme_scorer(each, inp_keys, mask_token):
    score = 0
    #print(inp_keys)
    for each_key in inp_keys:
        if each_key != mask_token:
            #print(each_key)
            if G.has_edge(each, each_key):
                score += G[each][each_key]['weight']
    return score / len(inp_keys)



all_possible_rhymes = [each2 for each in rhymes for each2 in each]


def find_rhyme(inp, rnd=False, rhymes = rhymes, inp_keys=[]):
    
    if rnd == False:
        for r, each_row in enumerate(rhymes):
            for w, each_word in enumerate(each_row):

                if each_word == inp:

                    #print('inp ', inp)
                    each_row_tmp = each_row[:w] + each_row[w + 1:]
                    each_row_tmp = [[each, rhyme_scorer(inp, each)] for each in each_row_tmp]
                    each_row_tmp = sorted(each_row_tmp, key=lambda x:x[1], reverse=True)
                    if len(each_row_tmp) > 0:
                        #print(each_row_tmp)
                        return each_row_tmp[0][0]
                    else:
                        return inp
    else:
        if len(inp_keys)>0:
            

            if len(inp_keys[0])>0:
                #print('hello')
                
                all_possible_rhymes_scored = [[each, rnd_rhyme_scorer(each, inp_keys[0], mask_token)] for each in all_possible_rhymes]
                all_possible_rhymes_scored = sorted(all_possible_rhymes_scored, key=lambda x:x[1], reverse=True)
                #print(all_possible_rhymes_scored)


                return all_possible_rhymes_scored[0][0]
    return inp

def mutual_check(idx, final_set, comb4, comb3, mask_token):
    if idx == 1:
        return comb4[0]
    if idx % 2 == 1:
        # its odd (right-side)
        final_set_words_used = [each[0] for each in final_set]
        comb4_scored = []
        for candidate_list in comb4:
            candidate_list_cleaned = [each_word for each_word in candidate_list[0] if each_word != mask_token]
            new_score_tmp = candidate_list[-1]
            for each in final_set_words_used:
                each_cleaned = [each_word for each_word in each if each_word != mask_token]
                mutual_num = len([w for w in candidate_list_cleaned if w in each_cleaned])
                new_score = candidate_list[-1] * (1 - (mutual_num / 4))
                if new_score < new_score_tmp:
                    new_score_tmp = new_score
            comb4_scored.append(candidate_list + [new_score_tmp])
        comb4_scored_sorted = sorted(comb4_scored, key=lambda x:x[-1], reverse=True)
        return comb4_scored_sorted[0][:-1]
    else:
        # its even (left-side)
        final_set_words_used = [each[0] for each in final_set]
        comb3_scored = []
        for candidate_list in comb3:
            candidate_list_cleaned = [each_word for each_word in candidate_list[0] if each_word != mask_token]
            new_score_tmp = candidate_list[-1]
            for each in final_set_words_used:
                each_cleaned = [each_word for each_word in each if each_word != mask_token]
                mutual_num = len([w for w in candidate_list_cleaned if w in each_cleaned])
                new_score = candidate_list[-1] * (1 - (mutual_num / 3))
                if new_score < new_score_tmp:
                    new_score_tmp = new_score
            comb3_scored.append(candidate_list + [new_score_tmp])
        comb3_scored_sorted = sorted(comb3_scored, key=lambda x:x[-1], reverse=True)
        return comb3_scored_sorted[0][:-1]



def final_sequence(all_preds, candidate_seq, counter, format, mask_token, random = True):
    #print(candidate_seq)
    if format == 'M':
        # FEDIT
        last_non_mask_word_idx = max([index for index, element in enumerate(candidate_seq) if element != mask_token])
        if (counter + 1) % 2 == 1:
            if last_non_mask_word_idx >= 8:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1])
            else:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1]+ [mask_token, find_rhyme('', rnd=random, inp_keys=candidate_seq)])
        else:
            if candidate_seq[0][7] == mask_token:
                last_ghafie = all_preds[-1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                sequence = ' '.join(candidate_seq[0][:8] + [find_rhyme(last_ghafie)])
            else:
                last_ghafie = all_preds[-1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                sequence = ' '.join(candidate_seq[0][:8] + [mask_token, find_rhyme(last_ghafie)])

    elif format == 'GH':
        last_non_mask_word_idx = max([index for index, element in enumerate(candidate_seq) if element != mask_token])
        if (counter + 1) % 2 == 1:
            if last_non_mask_word_idx >= 8:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1])
            else:
                sequence = ' '.join(candidate_seq[0][:7]+ [mask_token, find_rhyme('', rnd=random, inp_keys=candidate_seq)])
        else:
            if counter == 1:
                if candidate_seq[0][7] == mask_token:
                    last_ghafie = all_preds[-1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                    sequence = ' '.join(candidate_seq[0][:8] + [find_rhyme('', rnd=random, inp_keys=candidate_seq)])
                else:
                    last_ghafie = all_preds[-1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                    sequence = ' '.join(candidate_seq[0][:8] + [mask_token, find_rhyme('', rnd=random, inp_keys=candidate_seq)])
            else:
                if candidate_seq[0][7] == mask_token:
                    last_ghafie = all_preds[1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                    sequence = ' '.join(candidate_seq[0][:8] + [find_rhyme(last_ghafie)])
                else:
                    last_ghafie = all_preds[1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                    sequence = ' '.join(candidate_seq[0][:8] + [mask_token, find_rhyme(last_ghafie)])
    elif format == 'D':
        last_non_mask_word_idx = max([index for index, element in enumerate(candidate_seq) if element != mask_token])
        if counter == 0:
            if last_non_mask_word_idx >= 8:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1])
            else:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1]+ [mask_token, find_rhyme('', rnd=random, inp_keys=candidate_seq)])
        else:
            if candidate_seq[0][7] == mask_token:
                last_ghafie = all_preds[-1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                sequence = ' '.join(candidate_seq[0][:8] + [find_rhyme(last_ghafie)])
            else:
                last_ghafie = all_preds[-1][list(all_preds[-1].keys())[-1]][0].split()[-1]
                sequence = ' '.join(candidate_seq[0][:8] + [mask_token, find_rhyme(last_ghafie)])
    elif format == 'R':
        last_non_mask_word_idx = max([index for index, element in enumerate(candidate_seq) if element != mask_token])
        if (counter + 1) % 2 == 1:
            if last_non_mask_word_idx >= 8:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1])
            else:
                sequence = ' '.join(candidate_seq[0][:last_non_mask_word_idx + 1]+ [mask_token, find_rhyme('', rnd=random, inp_keys=candidate_seq)])
        else:
            if candidate_seq[0][7] == mask_token:
                last_ghafie = all_preds[0][list(all_preds[-1].keys())[-1]][0].split()[-1]
                sequence = ' '.join(candidate_seq[0][:8] + [find_rhyme(last_ghafie)])
            else:
                last_ghafie = all_preds[0][list(all_preds[-1].keys())[-1]][0].split()[-1]
                sequence = ' '.join(candidate_seq[0][:8] + [mask_token, find_rhyme(last_ghafie)])        
    return sequence

In [ ]:
# find_rhyme(inp = '',
 #rnd=True, 
# inp_keys=[['وین', mask_token, mask_token, 'تعجب', 'حیرت', 'انبیا', mask_token, mask_token, mask_token, mask_token], 5, 0.675, 0.827398570285026])

In [ ]:
# Final heuriscs function ------------------------------------------

def heuristic(poetry, mask_token, numOfKeywords, mesra_first_elements_num, 
              mesra_second_elements_num):

    # first get the output of translator and clean it----------------
    mapped = [clean_poetry(poetry)]
    
    # keywords---------------------------------------------------------
    single_keys = extract_single_keys(mapped, numOfKeywords)
    joint_keys = extract_joint_keys(mapped, numOfKeywords)
    final_keys = extract_all_keys(single_keys, joint_keys, numOfKeywords)
    fks = calculate_fks(single_keys, joint_keys, final_keys)
    

    # finding syns and their positions-----------------------------
    keys = list(final_keys)
    synonyms = find_synonyms(keys, fks)
    keys_synonyms = find_keys_synonyms(keys, synonyms)
    key_poses, keys_synonyms = find_key_poses(keys_synonyms)
    print(colored('\nkey: ', 'red'), keys)
    print(colored('keys_synonyms: ', 'red'), keys_synonyms)
    

    #print('key_poses:', key_poses)
    indices_suggestions = find_indices_suggestions(key_poses)

    # number of beits-----------------------------------------
    beit_num = mean([len(each) for each in list(indices_suggestions.values())])

    beit_num = math.ceil(beit_num)
    if beit_num>4:
        beit_num = 4
    mesra_num = beit_num * 2

    # num of keys required------------------------------------
    fixed_num_keys_required = int(((mesra_num / 2) * mesra_first_elements_num) + ((mesra_num / 2) * mesra_second_elements_num))

    # If the number of keywords that we 
    # already have is more than the number of keywords
    #  we should have: 
    # we remove the least frequent ones
    # from the most keyword-aggregated indices 

    if fixed_num_keys_required < len(keys_synonyms):
        difference = len(keys_synonyms) - fixed_num_keys_required
        # print(difference, 'more than needed.')
        deletion_noms = sorted([[key, len(value)] for key, value in list(indices_suggestions.items())], key = lambda x: x[1], reverse=True)
        deletion_noms = deletion_noms[:difference]

        for ii in range(len(deletion_noms)):
            each = deletion_noms[0]
            idx = each[0]
            deletion_noms_per_idx = sorted([[key, all_fdist[idx][key]] for key in indices_suggestions[idx]], key = lambda x: x[1], reverse=False)
            #print()
            #print('deletion_noms', deletion_noms)
            #print('deletion_noms_per_idx', deletion_noms_per_idx)
            #print('indices_suggestions', indices_suggestions)
            #print('keys_synonyms', keys_synonyms)
            #print()
            keys_synonyms, key_poses, indices_suggestions = remove_keyword(deletion_noms_per_idx[0][0], keys_synonyms)
            deletion_noms = sorted([[key, len(value)] for key, value in list(indices_suggestions.items())], key = lambda x: x[1], reverse=True)
            deletion_noms = deletion_noms[:difference]


    # If the number of keywords that
    # we already have is less than the number of keywords 
    # we should have: 
    # we add synonyms of the most frequent keywords in 
    # the least keyword-aggregated indices 


    # Mitra has modified > to > ----------------------------------------------
    
    elif fixed_num_keys_required >= len(keys_synonyms):
        difference = fixed_num_keys_required - len(keys_synonyms)
        # print(difference, 'less than needed.')
        # print('indices_suggestions:', indices_suggestions)
        # addition_noms = sorted([[key, len(value)] for key, value in list(indices_suggestions.items()) if len(value) > 0], key = lambda x: x[1], reverse=False)
        # # print('addition_noms before truncating:', addition_noms)
        # addition_noms = addition_noms[:difference]
        # # print('after:', addition_noms)
        # # print('addition_noms:', addition_noms)
        # for each in addition_noms:
        #     idx = each[0]
        #     # print('idx:', idx)
            
        #     addition_noms_per_idx = sorted([[key, all_fdist[idx][key]] for key in indices_suggestions[idx]], key = lambda x: x[1], reverse=True)
        #     # print(colored('adding', 'blue'), addition_noms_per_idx)
        #     #for m in range(len()):
        #      #   keys_synonyms, key_poses, indices_suggestions = add_keyword(addition_noms_per_idx[0][0])

        addition_noms = sorted([[key, len(value)] for key, value in list(indices_suggestions.items()) if len(value) > 0], key = lambda x: x[1], reverse=False)
        addition_noms = addition_noms[:difference]

        for ii in range(len(addition_noms)):
            each = addition_noms[0]
            idx = each[0]
            addition_noms_per_idx = sorted([[key, all_fdist[idx][key]] for key in indices_suggestions[idx]], key = lambda x: x[1], reverse=True)
            #print()
            #print('addition_noms', addition_noms)
            #print('addition_noms_per_idx', addition_noms_per_idx)
            #print('indices_suggestions', indices_suggestions)
            #print('keys_synonyms', keys_synonyms)
            #print()
            keys_synonyms, key_poses, indices_suggestions = add_keyword(addition_noms_per_idx[0][0], keys_synonyms)
            addition_noms = sorted([[key, len(value)] for key, value in list(indices_suggestions.items()) if len(value) > 0], key = lambda x: x[1], reverse=False)
            addition_noms = addition_noms[:difference]
        
    # finding possible combinitions-----------------------
    # Let's find out all possible combinations of keywords among the 10 indices
    # In order not to necessarily choose a keyword from an index, we add 'NONE' to all indices
    indices_contents = []
    for each in indices_suggestions.items():
        indices_contents.append(each[1] + [mask_token])
    
    
    # Here we calculate all possible combinations
    possible_combinations = list(product(*indices_contents))
    print(colored('\nlen(possible_combinations) = ', 'red'), len(possible_combinations))


    # Here we choose those combinations in which mesra_first_elements_num (4) 
    # and mesra_second_elements_num (3) number of keywords are seen
    possible_combinations_4, possible_combinations_3 = [], []
    for each in possible_combinations:
        each = list(each)
        if each.count(mask_token) == (10 - mesra_first_elements_num):
            possible_combinations_4.append(each)
        if each.count(mask_token) == (10 - mesra_second_elements_num):   
            possible_combinations_3.append(each) 
    
    
    # Let's measure d, g, and sig for each combination
    possible_combinations_4_rated = {}
    for i in range(len(possible_combinations_4)):
        # info_dict contains d (index = 0) and g (index = 1) and sig (index = 2)
        d = calculate_d(possible_combinations_4[i], mask_token)
        g = calculate_g(possible_combinations_4[i], mask_token)
        sig = sigmoid(math.log(d, 2) * g)
        info_dict = [d, g, sig]
        possible_combinations_4_rated[i] = info_dict

    possible_combinations_3_rated = {}
    for i in range(len(possible_combinations_3)):
        # info_dict contains d (index = 0) and g (index = 1) and sig (index = 2)
        d = calculate_d(possible_combinations_3[i], mask_token)
        g = calculate_g(possible_combinations_3[i], mask_token)
        sig = sigmoid(math.log(d, 2) * g)
        info_dict = [d, g, sig]
        possible_combinations_3_rated[i] = info_dict

    # Let's zip both the suggestions and their respective d, g, and sig.
    final_combinations_4 = []
    for i in range(len(possible_combinations_4)):
        d, g, sig = possible_combinations_4_rated[i]
        final_combinations_4.append([possible_combinations_4[i], d, g, sig])

    final_combinations_3 = []
    for i in range(len(possible_combinations_3)):
        d, g, sig = possible_combinations_3_rated[i]
        final_combinations_3.append([possible_combinations_3[i], d, g, sig])

    # Let's sort all combinations by their sigmoid value
    
    final_combinations_4_sorted =  sorted(final_combinations_4, key = lambda x: x[-1], reverse=True)
    final_combinations_3_sorted =  sorted(final_combinations_3, key = lambda x: x[-1], reverse=True)

    # Let's make our the_final_set which is made up of top 4 and 3 combinations.
    the_final_set = []
    if len(final_combinations_4_sorted) + len(final_combinations_3_sorted) < mesra_num or len(final_combinations_4_sorted)  == 0 or len(final_combinations_3_sorted) == 0:
        
        the_final_set = []
        for i in range(len(possible_combinations)):
            d = calculate_d(possible_combinations[i], mask_token)
            g = calculate_g(possible_combinations[i], mask_token)
            try:
                sig = sigmoid(math.log(d, 2) * g)
            except:
                sig = 0
            the_final_set.append([list(possible_combinations[i]), d, g, sig])
            
        the_final_set =  sorted(the_final_set, key = lambda x: x[-1], reverse=True)

    else:
        for i in range(mesra_num):
            the_final_set.append(mutual_check(i + 1, the_final_set, final_combinations_4_sorted, final_combinations_3_sorted, mask_token)) 



    # output masked poetry
    # poetries_with_msk = [' '.join(each[0]) for each in the_final_set]
    #poetries_with_msk = [each[0] for each in the_final_set]

    return the_final_set


In [ ]:
# FEDIT
def rhyme_scorer(inp, candidate):
    if G_ghafie.has_edge(inp, candidate):
        return G_ghafie[inp][candidate]['weight']
    else:
        return 0



In [ ]:
type(list(poetryBeam['F'].values()))

In [ ]:
for i in list(poetryBeam['B'].keys())[:20]:
    print(i)

In [ ]:
def underline_join(words):
    return '_'.join(words)

def beam_scorer(lastWords, poetryBeam):
    score = 0
    if len(lastWords) == 4:
        sequence = underline_join(lastWords[:4])
        if sequence in poetryBeam['F']:
            score += 3 * poetryBeam['F'][sequence]
        sequence = underline_join(lastWords[:3])
        if sequence in poetryBeam['T']:
            score += 2 * poetryBeam['T'][sequence]
        sequence = underline_join(lastWords[:2])
        if sequence in poetryBeam['B']:
            score += 1 * poetryBeam['B'][sequence]
    elif len(lastWords) == 3:
        sequence = underline_join(lastWords[:3])
        if sequence in poetryBeam['T']:
            score += 4 * poetryBeam['T'][sequence]
        sequence = underline_join(lastWords[:2])
        if sequence in poetryBeam['B']:
            score += 2 * poetryBeam['B'][sequence]
    elif len(lastWords) == 2:    
        sequence = underline_join(lastWords[:2])
        if sequence in poetryBeam['B']:
            score += 6 * poetryBeam['B'][sequence]
    return score/6

# Phase ||| MLM

In [ ]:
def import_model(model_name='BERT', on='beit', is_roberta = False):


    """
    import the models based on their checkpoints and training parameters
    """
    

    if on == 'beit':
        if model_name=='BERT V2':
            model_folder_path = r'.../Pretrained Models/Pretrained on beit/BERT_V2_0.4_beit/'
        elif  model_name=='BERT V3':
            model_folder_path = r'.../Pretrained Models/Pretrained on beit/BERT_0.4_beit/'
        elif  model_name=='Roberta':
            is_roberta = True
            model_folder_path = '.../Pretrained Models/Pretrained on beit/Roberta_0.4_beit/'
        elif  model_name=='Distilbert':
            model_folder_path = '.../Pretrained Models/Pretrained on beit/DistilBERT_0.4_beit/'
        elif  model_name=='Albert':
            model_folder_path = '.../Pretrained Models/Pretrained on beit/Albert_0.4_beit'
        
        
        


    else:


        if model_name=='BERT V3':

            model_folder_path = '.../Pretrained Models/Pretrained on verses/BERT_0.15_Verse/with 5 epochs/'
            
        elif  model_name=='Albert':

            model_folder_path = '.../Pretrained Models/Pretrained on verses/Albert_0.15_Verse/with 5 epochs/'
            
        elif  model_name=='Roberta':

            is_roberta = True
            model_folder_path = '.../Pretrained Models/Pretrained on verses/Roberta_0.15_Verse/with 5 epochs/'
            
        elif  model_name=='Distilbert':

            model_folder_path = '.../Pretrained Models/Pretrained on verses/DistilBERT_0.15_Verse/with 5 epochs/'
            


    model = AutoModelForMaskedLM.from_pretrained(model_folder_path)
    tokenizer = AutoTokenizer.from_pretrained(model_folder_path, use_fast=True)
    mask_token = tokenizer.mask_token

    return mask_token, model, tokenizer, is_roberta


In [ ]:
def get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens):


    """
    disallowing the model to generate very short tokens and punctuations
    """
    unw = []

    for v in tokenizer.vocab.keys():
        if len(v)<2:
            #print(v)
            unw.append(v)

            
    banned_tokens = ['«', ':', '،', '/', '*', ']', '[', '؟', '…', 'ی', tokenizer.unk_token] + unw
    # these two have meanings in farsi so we won't remove them
    # و = and 
    # ز = from
    all_banned_tokens = list(set(banned_tokens).union(all_unw_tokens)-set(['ز', 'و']))
    all_banned_tokens = [i.strip() for i in all_banned_tokens] 

    if is_roberta == True :
        all_banned_tokens = all_banned_tokens + ['/ ', ' /']
                                


    banned_ids = []

    for i in tokenizer.batch_encode_plus(all_banned_tokens, add_special_tokens=False).input_ids:
        if len(i)==1:
            banned_ids.append(i[0])

        elif len(i)>1:
            banned_ids.append(i[1])

    print('Total length of unwanted tokens: ', len(banned_ids))

    return banned_ids

    

# Final Function


In [ ]:
# importing from the translator

translator_output = pd.read_csv('.../Results/MultiHeadAttention_Poetry_17_withAugmented_P&T.csv') 

translator_output.dropna(inplace=True)
translator_output.reset_index(inplace=True, drop=True)

poetry_mha = translator_output.loc[:, 'poetry_generated_MHA'].values.tolist()
poetry_gt = translator_output.loc[:, 'poetry_ground_truth'].values.tolist()

for i in poetry_mha:
    if type(i) != str:
        print(i)

In [ ]:

def get_transformer_suggestions(sequence, model, tokenizer, banned_ids, top_k=5):

    #sequence = mask_seq(sequence, model, tokenizer)

    ids_main = tokenizer.encode(sequence,
                            return_tensors="pt",
                            add_special_tokens=True)
    #print(ids_main)
    ids_ = ids_main.detach().clone()
    position = torch.where(ids_main == tokenizer.mask_token_id)

    positions_list = position[1].numpy().tolist()



    suggestion = []
    #print(positions_list)
    model_logits = model(ids_)['logits'][0][positions_list[0]]

    # setting inf to the '/' token
    
    #print(model_logits)
    model_logits[banned_ids] = -math.inf
    
    
    top_k_tokens = torch.topk(model_logits, top_k, dim=0).indices.tolist()
    
    for j in range(len(top_k_tokens)):

        suggestion.append(tokenizer.decode(top_k_tokens[j]))

    
    return suggestion      

In [ ]:

# BERT ADDED


def beam_search(raw_sentence , model, tokenizer, banned_ids, beam_depth=3, 
                top_k=5, mask_token='[MASK]', end_token='<END>', start_token='<START>'):

    raw_sentence_for_bert = raw_sentence.replace('<SEP>', '/')

    leadings = [[[start_token], 0]]
    mask_counter = 0

    #print(tokenizer_bert.tokenize(raw_sentence))
    for i, step in enumerate(raw_sentence.split()[1:]):


        #print('STEP', step)
        if step == mask_token:
            all_candidates = []
            for lead in leadings:
                lead_thread, lead_score = lead

                lead_thread = [m.strip() for m in lead_thread]


                # BERT -----------------------------------------------------

                # change this part when using distilbert
                # mask_seq
                #print(colored(i, 'green'))
                if i == 0: 
                    transformer_inp = ' '.join(raw_sentence_for_bert.split()[i+1:-1])
                    #print('BERT INPUT', transformer_inp)

                else :
                    #print('lead_thread', lead_thread)
                    transformer_inp = ' '.join(lead_thread[1:] + raw_sentence_for_bert.split()[i+1:-1])
                    #print('BERT INPUT', transformer_inp)
                #print(colored('\nBERT INPUT: ', 'green'), transformer_inp)
                
                candidates = get_transformer_suggestions(transformer_inp,
                            model= model, 
                            tokenizer=tokenizer, 
                            top_k=5,
                            banned_ids=banned_ids)
                
                candidates = [ca.strip() for ca in candidates]
                #print('candidates', candidates)

                # ------------------------------------------------------------
                for j, cand in enumerate(candidates):

                    new_thread = lead_thread[-3:] + [cand]
                    #print(new_thread)
                    new_score = beam_scorer(new_thread, poetryBeam)
                    #print('new_score', new_score)
                    # new_score = j


                    #print('cand:{} score: {}'.format(cand, new_score))
                    all_candidates.append([lead_thread + [cand], new_score])
                    #print(candidates)

                    all_candidates = sorted(all_candidates, key=lambda x:x[1], reverse=True)
                    #print('all_candidates sorted ', all_candidates)
                    #print(new_thread, new_score)

            # print('old leadings', leadings)
            leadings = all_candidates[:beam_depth]
            # print('all_candidates', all_candidates)
            # print('new leadings', leadings)

            mask_counter += 1
        elif step == end_token:
            new_score = 1
            # print('old leadings', leadings)
            for i in range(len(leadings)):
                # print(each_lead[0][-3:] + ['<END>'])
                new_score = beam_scorer(each_lead[0][-3:] + [end_token], poetryBeam)
                # new_score = 1
                leadings[i][1] += new_score
                leadings = sorted(leadings, key=lambda x:x[1], reverse=True)
            #print('new leadings', leadings)
        else:
            # append step to all leading threads
            for each_lead in leadings:
                each_lead[0].append(step)

    # print('\n', ' '.join(leadings[0][0]))
    return ' '.join(leadings[0][0][1:])


In [ ]:

def clean_mha(txt):

    txt = re.sub('<sep>', ' / ', txt)
    txt = re.sub('(<start>)|(<end>)', '', txt)

    return txt

def clean_fpv(txt):
    remove_ = [' *\[ZWNJ\] *', '<START>']
    txt = re.sub(' ##', ' ', txt)
    txt = re.sub(' <SEP> ', ' / ', txt)
    for patt in remove_:
        txt = re.sub(patt, ' ', txt)


    final_txt = [txt.split()[0]]

    for t in txt.split()[1:]:
        if t != final_txt[-1]:
            final_txt.append(t)
    

    return ' '.join(final_txt)

In [ ]:

results_dict = {
    'poetry_format': [],
    'numOfKeywords': [],
    'mesra_first_elements_num': [],
    'mesra_second_elements_num' : [],
    'address': [],
    'beam_depth': []
}



In [ ]:
def fill_masks_on_beit(model, tokenizer, translator_output,
               numOfKeywords, mesra_first_elements_num, 
                mesra_second_elements_num, banned_ids,
               m = 10, poetry_format = 'M', beam_depth = 3, 
               model_name ='Distilbert',
               output_folder = '.../Results/Experiment 2/DistilBERT/'
):
    
    
    """
    create the output as a csv file with the
    ground truth
    poetry generated by the multiheadattention translator
    poetry generated after choosing keywords and filling the masks
    the output of heuristics
    """


    # define the output format and address

    address = output_folder +f'Final_{model_name}_{poetry_format}_poetry_format_{numOfKeywords}_numOfKeywords_{mesra_first_elements_num}_mesra_first_elements_num_{mesra_second_elements_num}_mesra_second_elements_num_{beam_depth}_beam_depth_{m}_first.csv'

    results_dict['numOfKeywords'].append(numOfKeywords)
    results_dict['mesra_first_elements_num'].append(mesra_first_elements_num)
    results_dict['mesra_second_elements_num'].append(mesra_second_elements_num)
    results_dict['address'].append(address)
    results_dict['poetry_format'].append(poetry_format)
    results_dict['beam_depth'].append(beam_depth)

    df_output_beit = {
        'poetry_ground_truth': poetry_gt[:m],
        'poetry_generated_MHA': poetry_mha[:m],
        'final_predicted_verses' : [],
        'heuristics': []
        }

    # go through the dataset

    mask_token = tokenizer.mask_token

    for i in tqdm(range(0, len(poetry_mha[:m])), position=0, leave=True):
        
        # first get the heuristic masked seqs
        masked_sequences = heuristic(poetry_mha[i], mask_token, 
                                     numOfKeywords, mesra_first_elements_num, 
                                     mesra_second_elements_num)
        #print('after masked seq')


        # processing each verse and them giving the whole
        # beit to transformers and beamsearch
        all_preds, all_preds_tmp = [], []

        for j in range(int(len(masked_sequences) / 2)):
        
            j *= 2

            m1, m2 = masked_sequences[j], masked_sequences[j+1]


            try:
                # verse 1
                m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m1_final]})

                # verse 2
                m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m2_final]})


                
                all_preds.append(beam_search('<START> ' + m1_final + ' <SEP> ' + m2_final + ' <END>', 
                                            beam_depth = beam_depth, model=model, tokenizer=tokenizer, top_k=5,
                                            banned_ids = banned_ids, mask_token=mask_token,
                                             end_token='<END>', start_token='<START>'))

                

            except: 

                m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m1_final]})

                m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m2_final]})

                print(colored('\nEXCEPTION!!!', 'red'))

                print([ ' '.join(i[0]) for i in masked_sequences])
                
                all_preds.append('Not a sequence')
                df_output_beit['heuristics'].append(m1_final + ' / ' + m2_final)
            
            

        df_output_beit['heuristics'].append([m[0].replace(mask_token, '_') for m in [list(n.values())[0] for n in all_preds_tmp]])
        
        df_output_beit['final_predicted_verses'].append(all_preds)

        print('\n')
        print(colored('heuristics: ', 'blue'), df_output_beit['heuristics'][i])
        print(colored('all_preds: ', 'green'), df_output_beit['final_predicted_verses'][i])



    # cleaning the final file

    df_output_beit = pd.DataFrame(df_output_beit)
    df_output_beit.loc[:, 'final_predicted_verses'] = df_output_beit.loc[:, 'final_predicted_verses'].apply(lambda x : [clean_fpv(i) for i in x])


    df_output_beit.loc[:, 'poetry_generated_MHA'] = df_output_beit.loc[:, 'poetry_generated_MHA'].apply(lambda x: clean_mha(x))

    
    df_output_beit.to_csv(address, index=False)

    return df_output_beit, address


In [ ]:
def fill_masks_on_verse(model, tokenizer, translator_output,
                        numOfKeywords, mesra_first_elements_num, 
                        mesra_second_elements_num, banned_ids,
                        m = 10, poetry_format = 'M', beam_depth = 2, 
                        model_name ='Distilbert', 
                        output_folder = '.../Results/Experiment 2/DistilBERT/'):
    
    
    """
    generate tokens for masks in each verse
    """


    # define the output format and address

    address = output_folder +f'Final_{model_name}_Verse_{poetry_format}_poetry_format_{numOfKeywords}_numOfKeywords_{mesra_first_elements_num}_mesra_first_elements_num_{mesra_second_elements_num}_mesra_second_elements_num_{beam_depth}_beam_depth_{m}_first.csv'

    results_dict['numOfKeywords'].append(numOfKeywords)
    results_dict['mesra_first_elements_num'].append(mesra_first_elements_num)
    results_dict['mesra_second_elements_num'].append(mesra_second_elements_num)
    results_dict['address'].append(address)
    results_dict['poetry_format'].append(poetry_format)
    results_dict['beam_depth'].append(beam_depth)

    df_output_beit = {
        'poetry_ground_truth': poetry_gt[:m],
        'poetry_generated_MHA': poetry_mha[:m],
        'final_predicted_verses' : [],
        'heuristics': []
        }

    # go through the dataset

    mask_token = tokenizer.mask_token

    for i in tqdm(range(0, len(poetry_mha[:m])), position=0, leave=True):
        
        # first get the heuristic masked seqs
        masked_sequences = heuristic(poetry_mha[i], mask_token, 
                                     numOfKeywords, mesra_first_elements_num, 
                                     mesra_second_elements_num)
        #print('after masked seq')


        # processing each verse and them giving the whole
        # beit to transformers and beamsearch
        all_preds, all_preds_tmp = [], []

        for j in range(int(len(masked_sequences) / 2)):
        
            j *= 2

            m1, m2 = masked_sequences[j], masked_sequences[j+1]


            # verse 1
            m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format, mask_token)
            all_preds_tmp.append({'0':[m1_final]})

            # verse 2
            m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format, mask_token)
            all_preds_tmp.append({'0':[m2_final]})

            pred_v_1 = beam_search(raw_sentence= '<START> ' + m1_final + ' <SEP> ',
                                    beam_depth=beam_depth,
                                    model=model,
                                    tokenizer=tokenizer, 
                                    top_k=5,
                                    banned_ids = banned_ids,
                                    mask_token=mask_token,
                                    end_token='<SEP>', 
                                    start_token='<START>')
            
            pred_v_2 = beam_search(raw_sentence= '<SEP> ' + m2_final + ' <END>',
                                    beam_depth = beam_depth,
                                    model=model,
                                    tokenizer=tokenizer, 
                                    top_k=5,
                                    banned_ids = banned_ids,
                                    mask_token=mask_token,
                                    end_token='<END>',
                                    start_token='<SEP>')
            
            #print('\npred_v_1', pred_v_1)
            #print('pred_v_2', pred_v_2)
            all_preds.append(pred_v_1 + ' / ' + pred_v_2)
                #all_preds.append()
                

                



        df_output_beit['heuristics'].append([m[0].replace(mask_token, '_') for m in [list(n.values())[0] for n in all_preds_tmp]])
        
        df_output_beit['final_predicted_verses'].append(all_preds)

        print('\n')
        print(colored('heuristics: ', 'blue'), df_output_beit['heuristics'][i])
        print(colored('all_preds: ', 'green'), df_output_beit['final_predicted_verses'][i])



    # cleaning the final file

    df_output_beit = pd.DataFrame(df_output_beit)
    df_output_beit.loc[:, 'final_predicted_verses'] = df_output_beit.loc[:, 'final_predicted_verses'].apply(lambda x : [clean_fpv(i) for i in x])


    df_output_beit.loc[:, 'poetry_generated_MHA'] = df_output_beit.loc[:, 'poetry_generated_MHA'].apply(lambda x: clean_mha(x))

    
    df_output_beit.to_csv(address, index=False)

    return df_output_beit, address







In [ ]:
# beam search example after importing the model

beam_search(
    raw_sentence = f'دامانها {tokenizer.mask_token} {tokenizer.mask_token} فشاند جانها {tokenizer.mask_token} {tokenizer.mask_token} {tokenizer.mask_token} {tokenizer.mask_token} {tokenizer.mask_token} <SEP> <END>',
    beam_depth=2,
    model=model,
    tokenizer= tokenizer,
    top_k=5,
    banned_ids = banned_ids, mask_token = tokenizer.mask_token
    )

In [ ]:
models = ['BERT V2', 'BERT V3', 'Albert', 'Distilbert', 'Roberta']

# Distilbert


In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='Distilbert', on='beit', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)

In [ ]:
out_, address = fill_masks_on_beit(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            numOfKeywords=20,
            mesra_first_elements_num=4, 
            mesra_second_elements_num=4, banned_ids = banned_ids,
            m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'Distilbert_gh',
            output_folder = '.../Results/Experiment 2/DistilBERT/'
)

In [ ]:
out_ # 20 key num

In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='Distilbert', on='verse', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)
gc.collect()

In [ ]:
out_d_v, address_d_v = fill_masks_on_verse(
    model=model, tokenizer=tokenizer, 
    translator_output=translator_output,
    numOfKeywords=20,
    mesra_first_elements_num=4, 
    mesra_second_elements_num=4, banned_ids = banned_ids,
    m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'DistilBERT_gh',
    output_folder = '.../Results/Experiment 2/DistilBERT')

In [ ]:
out_d_v

In [ ]:
address_d_v

In [ ]:
out_d_v.to_csv('.../Results/Experiment 2/DistilBERT/Final_DistilBERT_gh_Verse_M_poetry_format_20_numOfKeywords_4_mesra_first_elements_num_4_mesra_second_elements_num_3_beam_depth_100_first.csv', index=False)

# BERT V2

In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='BERT V2', on='beit', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)

In [ ]:
out__b_b, address_b_b = fill_masks_on_beit(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            numOfKeywords=20,
            mesra_first_elements_num=4, 
            mesra_second_elements_num=4, banned_ids = banned_ids,
            m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'BERT_V2_gh',
            output_folder = '.../Results/Experiment 2/Bert V2/'
)

In [ ]:
out_b.to_csv('.../Results/Experiment 2/Bert V2/Final_BERT_V2_M_poetry_format_20_numOfKeywords_4_mesra_first_elements_num_4_mesra_second_elements_num_100_first.csv', index=False)

# BERT V3

In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='BERT V3', on='beit', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)

In [ ]:
out_b_3_b, address_b_3_b = fill_masks_on_beit(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            numOfKeywords=20,
            mesra_first_elements_num=4, 
            mesra_second_elements_num=4, banned_ids = banned_ids,
            m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'BERT_V3_gh',
            output_folder = '.../Results/Experiment 2/Bert V3/'
)

In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='BERT V3', on='verse', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)
gc.collect()

In [ ]:
out_b_v, address_b_v = fill_masks_on_verse(
    model=model, tokenizer=tokenizer, 
    translator_output=translator_output,
    numOfKeywords=20,
    mesra_first_elements_num=4, 
    mesra_second_elements_num=4, banned_ids = banned_ids,
    m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'BERT_V3_gh_last',
    output_folder = '.../Results/Experiment 2/Bert V3/')

In [ ]:
address_b_v

In [ ]:
out_b_v.to_csv(address_b_v, index=False)

In [ ]:
gc.collect()

# Roberta

In [ ]:
mask_token, model, tokenizer, is_roberta = import_model(model_name='Roberta', on='beit', is_roberta = True)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=is_roberta, all_unw_tokens=all_unw_tokens)

In [ ]:
out_r_v, address_r_b = fill_masks_on_beit(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            numOfKeywords=20,
            mesra_first_elements_num=4, 
            mesra_second_elements_num=4, banned_ids = banned_ids,
            m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'Roberta_gh',
            output_folder = '.../Results/Experiment 2/Roberta/'
)

In [ ]:
mask_token, model, tokenizer, is_roberta = import_model(model_name='Roberta', on='verse', is_roberta = True)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=is_roberta, all_unw_tokens=all_unw_tokens)

In [ ]:
out_r_v, address_r_v = fill_masks_on_verse(
    model=model, tokenizer=tokenizer, 
    translator_output=translator_output,
    numOfKeywords=20,
    mesra_first_elements_num=4, 
    mesra_second_elements_num=4, banned_ids = banned_ids,
    m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'Roberta_gh',
    output_folder = '.../Results/Experiment 2/Roberta/')

In [ ]:
out_r_v.to_csv(address_r_v, index=False)

# Albert


In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='Albert', on='beit', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)

In [ ]:
out_a, address = fill_masks_on_beit(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            numOfKeywords=20,
            mesra_first_elements_num=4, 
            mesra_second_elements_num=4, banned_ids = banned_ids,
            m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'Albert_gh',
            output_folder = '.../Results/Experiment 2/Albert/'
)

In [ ]:
out_a

In [ ]:
mask_token, model, tokenizer, Roberta = import_model(model_name='Albert', on='verse', is_roberta = False)
banned_ids = get_banned_tokens(model, tokenizer, is_roberta=False, all_unw_tokens=all_unw_tokens)

In [ ]:
out_a_v, address = fill_masks_on_verse(
    model=model, tokenizer=tokenizer, 
    translator_output=translator_output,
    numOfKeywords=20,
    mesra_first_elements_num=4,  banned_ids = banned_ids,
    mesra_second_elements_num=4,
    m = 100, poetry_format = 'M', beam_depth = 3, model_name = 'Albert_gh',
    output_folder = '.../Results/Experiment 2/Albert/')

# *Other experiments*

In [ ]:
m = 10
poetry_format = 'M'
beam_depth = 2
mask_token=tokenizer.mask_token

In [ ]:
m = 10

mask_token = tokenizer.mask_token
df_output_beit = {
    'poetry_ground_truth': poetry_gt[:m],
    'poetry_generated_MHA': poetry_mha[:m],
    'final_predicted_verses' : [],
    'heuristics': []
    }

for i in tqdm(range(0, len(poetry_mha[:m])), position=0, leave=True):
    
    
    # first get the heuristic masked seqs
    masked_sequences = heuristic(poetry_mha[i], mask_token)

    


# FEDIT
    all_preds, all_preds_tmp = [], []
    poetry_format = 'GH'
    beam_depth = 2

    
    for j in range(int(len(masked_sequences) / 2)):
        
        j *= 2
        m1, m2 = masked_sequences[j], masked_sequences[j+1]


        m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format, mask_token)
        all_preds_tmp.append({'0':[m1_final]})
        

        m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format, mask_token)
        all_preds_tmp.append({'0':[m2_final]})

        #print(m1_final)
        
        all_preds.append(beam_search('<START> ' + m1_final + ' / ' + m2_final + ' <END>', 
                                     beam_depth, model=model, tokenizer=tokenizer, top_k=5,
                                     banned_ids = banned_ids, mask_token=mask_token))
                




            

    df_output_beit['heuristics'].append([m[0].replace(mask_token, '_') for m in [list(n.values())[0] for n in all_preds_tmp]])
    
    df_output_beit['final_predicted_verses'].append(all_preds)

    print('\n')
    print(colored('heuristics: ', 'blue'), df_output_beit['heuristics'][i])
    print(colored('all_preds: ', 'green'), df_output_beit['final_predicted_verses'][i])
    


In [ ]:
df_output_beit = pd.DataFrame(df_output_beit)
df_output_beit.loc[:, 'final_predicted_verses'] = df_output_beit.loc[:, 'final_predicted_verses'].apply(lambda x : [clean_fpv(i) for i in x])

def clean_mha(txt):

    txt = re.sub('<sep>', ' / ', txt)
    txt = re.sub('(<start>)|(<end>)', '', txt)

    return txt

df_output_beit.loc[:, 'poetry_generated_MHA'] = df_output_beit.loc[:, 'poetry_generated_MHA'].apply(lambda x: clean_mha(x))

#df_output.loc[:, 'heuristics'] = df_output.loc[:, 'heuristics'].apply(lambda x: [i[0].replace(mask_token, '_') for i in x])
df_output_beit

In [ ]:
df_output_beit.to_csv('df_output_beit.csv')

In [ ]:
df_output_beit.to_csv('.../Results/df_output_beit.csv')

In [ ]:
m = 10


df_output_beit = {
    'poetry_ground_truth': poetry_gt[:m],
    'poetry_generated_MHA': poetry_mha[:m],
    'final_predicted_verses' : [],
    'heuristics': []
    }

heuristics = []

for i in tqdm(range(0, len(poetry_mha[:m])), position=0, leave=True):
    
    
    # first get the heuristic masked seqs
    masked_sequences = heuristic(poetry_mha[i])

    


# FEDIT
    all_preds, all_preds_tmp = [], []
    poetry_format = 'M'
    beam_depth = 2

    
    for j in range(int(len(masked_sequences) / 2)):
        
        j *= 2
        m1, m2 = masked_sequences[j], masked_sequences[j+1]


        try:
            
            m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format)
            all_preds_tmp.append({'0':[m1_final]})

            m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format)
            all_preds_tmp.append({'0':[m2_final]})


            
            all_preds.append(beam_search('<START> ' + m1_final + ' / ' + m2_final + ' <END>', beam_depth))
                



        except: 
            print(colored('\nEXCEPTION!!!', 'red'))

            print([ ' '.join(i[0]) for i in masked_sequences])
            
            all_preds.append('Not a sequence')
            df_output_beit['heuristics'].append(m1_final + ' / ' + m2_final)
            

            

    df_output_beit['heuristics'].append([m[0].replace(mask_token, '_') for m in [list(n.values())[0] for n in all_preds_tmp]])
    
    df_output_beit['final_predicted_verses'].append(all_preds)

    print('\n')
    print(colored('heuristics: ', 'blue'), df_output_beit['heuristics'][i])
    print(colored('all_preds: ', 'green'), df_output_beit['final_predicted_verses'][i])
    


## CONT

In [ ]:
def fill_masks(model, tokenizer, translator_output, output_folder,
               numOfKeywords, mesra_first_elements_num, 
                mesra_second_elements_num,
               m = 10, poetry_format = 'M', beam_depth = 2):
    
    
    """
    create the output as a csv file with the
    ground truth
    poetry generated by the multiheadattention translator
    poetry generated after choosing keywords and filling the masks
    the output of heuristics
    """


    # define the output format and address

    address = output_address +f'final_{poetry_format}_poetry_format_{numOfKeywords}_numOfKeywords_{mesra_first_elements_num}_mesra_first_elements_num_{mesra_second_elements_num}_mesra_second_elements_num.csv'

    

    df_output_beit = {
        'poetry_ground_truth': poetry_gt[:m+1],
        'poetry_generated_MHA': poetry_mha[:m+1],
        'final_predicted_verses' : [],
        'heuristics': []
        }

    # go through the dataset

    mask_token = tokenizer.mask_token

    for i in tqdm(range(0, len(poetry_mha[:m+1])), position=0, leave=True):
        
        # first get the heuristic masked seqs
        masked_sequences = heuristic(poetry_mha[i], mask_token, 
                                     numOfKeywords, mesra_first_elements_num, 
                                     mesra_second_elements_num)
        print(colored(i, 'green'))


        # processing each verse and them giving the whole
        # beit to transformers and beamsearch
        all_preds, all_preds_tmp = [], []
        #print(masked_sequences)

        for j in range(int(len(masked_sequences) / 2)):
        
            j *= 2

            m1, m2 = masked_sequences[j], masked_sequences[j+1]

            print('\nBefore')
            print('m1: ', m1[0])
            print('m2: ', m2[0])
            m1[0] = m1[0][:-3]
            m2[0] = m2[0][:-3]
            print('\nAfter')
            print('m1: ', m1[0])
            print('m2: ', m2[0])

            try:
                # verse 1
                m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m1_final]})

                # verse 2
                m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m2_final]})


                
                all_preds.append(beam_search('<START> ' + m1_final + ' <SEP> ' + m2_final + ' <END>', 
                                            beam_depth, model=model, tokenizer=tokenizer, top_k=5,
                                            banned_ids = banned_ids, mask_token=mask_token))

                

            except: 

                m1_final = final_sequence(all_preds_tmp, m1, j, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m1_final]})

                m2_final = final_sequence(all_preds_tmp, m2, j + 1, poetry_format, mask_token)
                all_preds_tmp.append({'0':[m2_final]})

                print(colored('\nEXCEPTION!!!', 'red'))

                print([ ' '.join(i[0]) for i in masked_sequences])
                
                all_preds.append('Not a sequence')
                df_output_beit['heuristics'].append(m1_final + ' / ' + m2_final)
            
            

        df_output_beit['heuristics'].append([m[0].replace(mask_token, '_') for m in [list(n.values())[0] for n in all_preds_tmp]])
        
        df_output_beit['final_predicted_verses'].append(all_preds)

        print('\n')
        print(colored('heuristics: ', 'blue'), df_output_beit['heuristics'][i])
        print(colored('all_preds: ', 'green'), df_output_beit['final_predicted_verses'][i])



        if i%10==0:
            
            df_output_beit_ = {
        'poetry_ground_truth': poetry_gt[:i+1],
        'poetry_generated_MHA': poetry_mha[:i+1],
        'final_predicted_verses' : df_output_beit['final_predicted_verses'],
        'heuristics': df_output_beit['heuristics']
        }
            df_output_beit_ = pd.DataFrame(df_output_beit_)
            #df_output_beit_.to_csv(address, index=False)

    df_output_beit_ = pd.DataFrame(df_output_beit)
    # cleaning the final file
    
    df_output_beit_.loc[:, 'final_predicted_verses'] = df_output_beit_.loc[:, 'final_predicted_verses'].apply(lambda x : [clean_fpv(i) for i in x])

    df_output_beit_.loc[:, 'poetry_generated_MHA'] = df_output_beit_.loc[:, 'poetry_generated_MHA'].apply(lambda x: clean_mha(x))

    df_output_beit_.to_csv(address, index=False)

    return df_output_beit_


In [ ]:
fill_masks(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            output_folder = output_folder,
            numOfKeywords=6,
            mesra_first_elements_num=3, 
            mesra_second_elements_num=3,
            m = 2, poetry_format = 'M', beam_depth = 3) # -2

In [ ]:
fill_masks(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            output_folder = output_folder,
            numOfKeywords=6,
            mesra_first_elements_num=3, 
            mesra_second_elements_num=3,
            m = 10, poetry_format = 'M', beam_depth = 3) # -1

In [ ]:
fill_masks(model=model, tokenizer=tokenizer, 
            translator_output=translator_output,
            output_folder = output_folder,
            numOfKeywords=6,
            mesra_first_elements_num=3, 
            mesra_second_elements_num=3,
            m = 2, poetry_format = 'M', beam_depth = 3) # -2

In [ ]:
for t in rhymes:
    for i in t:
        if i == 'بود':
            print(t)

In [ ]:
rhymes

In [ ]:
len(translator_output.loc[:, 'poetry_generated_MHA'])

In [ ]:
len(df_output)

In [ ]:
df_output.to_csv('.../Results/final_output_15epochs_10May_BeitBert_0.7_6epochs_augmented_P&T.csv',
                 index=False)

In [ ]:
df_output.to_csv('.../Results/final_output_15epochs_9May_BeitBert_0.7_6epochs_augmented_P&T.csv',
                 index=False)

In [ ]:
df_output =  pd.DataFrame({'ground_truth': poetry_gt[:154],
             'poetry_mha':poetry_mha[:154],
             'final_predicted_verses': df_output['final_predicted_verses'][:154],
             'heuristics': df_output['heuristics'][:154],
             }

             )

In [ ]:
len(df_output['final_predicted_verses'][:154])

In [ ]:
len(df_output['heuristics'][:154])

In [ ]:
def beam_search(raw_sentence, mask_suggestions, beam_depth):
    leadings = [[['<START>'], 0]]
    mask_counter = 0
    for i, step in enumerate(raw_sentence.split()[1:]):
        print('STEP', step)
        if step == mask_token:
            all_candidates = []
            for lead in leadings:
                lead_thread, lead_score = lead
                print('BERT INPUT', ' '.join(lead_thread + raw_sentence.split()[i + 1:-1]))
                candidates = mask_suggestions[mask_counter]
                for j, cand in enumerate(candidates):
                    new_thread = lead_thread[-3:] + [cand]
                    # new_score = beam_scorer(new_thread, poetryBeam)
                    new_score = j
                    all_candidates.append([lead_thread + [cand], new_score])
                    all_candidates = sorted(all_candidates, key=lambda x:x[1], reverse=True)
                    print(new_thread, new_score)
            print('old leadings', leadings)
            leadings = all_candidates[:beam_depth]
            print('all_candidates', all_candidates)
            print('new leadings', leadings)
            mask_counter += 1
        elif step == '<END>':
            new_score = 1
            print('old leadings', leadings)
            for i in range(len(leadings)):
                # new_score = beam_scorer(each_lead[-3:] + ['<END>'], poetryBeam)
                new_score = 1
                leadings[i][1] += new_score
                leadings = sorted(leadings, key=lambda x:x[1], reverse=True)
            print('new leadings', leadings)
        else:
            # append step to all leading threads
            for each_lead in leadings:
                each_lead[0].append(step)
    return ' '.join(leadings[0][0])


raw_sentence = '<START> hello my [MASK] how [MASK] <END>'
mask_suggestions = [
                    ['friend', 'wife', 'husband'],
                    ['he', 'she', 'Ali']
]
beam_depth = 2
print(beam_search(raw_sentence, mask_suggestions, beam_depth))

In [ ]:


# example

get_transformer_suggestions('سلام بر تو ای [MASK] خوب [MASK]',
                            model= model_bert, 
                            tokenizer=tokenizer_bert, 
                            top_k=5)

In [ ]:
def mask_seq(sequence, model, tokenizer=tokenizer_bert):
    if tokenizer.mask_token == '<mask>':
        return re.sub(tokenizer.mask_token, '<mask>', sequence)
    else: return sequence

In [ ]:
def predict_seqs_dict(sequence, model, tokenizer, top_k=5, order='right-to-left'):

    sequence = mask_seq(sequence, model, tokenizer)

    ids_main = tokenizer.encode(sequence,
                            return_tensors="pt",
                            add_special_tokens=False)

    ids_ = ids_main.detach().clone()
    position = torch.where(ids_main == tokenizer.mask_token_id)

    positions_list = position[1].numpy().tolist()

    if order =='left-to-right':
        positions_list.reverse()

    elif order=='random':
        random.shuffle(positions_list)

    # print(positions_list)
    predictions_ids = {}
    predictions_detokenized_sents = {}

    for i in range(len(positions_list)):
        predictions_ids[i] = []
        predictions_detokenized_sents[i] = []
        # where do we have mask?
        # print(i)


        
        # if it was the first prediction, 
        # just go on and predict the first predictions
        

        if i==0:
            model_logits = model(ids_main)['logits'][0][positions_list[0]]
            top_k_tokens = torch.topk(model_logits, top_k, dim=0).indices.tolist()
            
            for j in range(len(top_k_tokens)):
                #print(j)
                ids_t_ = ids_.detach().clone()
                ids_t_[0][positions_list[0]] = top_k_tokens[j]
                predictions_ids[i].append(ids_t_)
                
                pred = tokenizer.decode(ids_t_[0])
                predictions_detokenized_sents[i].append(pred)

                # append the sentences and ids of this masked
                # token


        # if we already have some predictions, go on and fill the rest
        # of the masks by continuing the previous predictions
        if i!=0:
            for pred_ids in predictions_ids[i-1]:
                
                # get the logits
                #print('previous pred:', pred_ids)
                model_logits = model(pred_ids)['logits'][0][positions_list[i]]
                # get the top 5 of this prediction and masked token
                top_k_tokens = torch.topk(model_logits, top_k, dim=0)\
                .indices.tolist()

                for top_id in top_k_tokens:
                    # print(top_id)
                    
                    ids_t_i = pred_ids.detach().clone()
                    ids_t_i[0][positions_list[i]] = top_id

                    #print(ids_t_i)

                    #print(ids_)
                    pred = tokenizer.decode(ids_t_i[0])
                    #print(pred)

                    # append the sentences and ids of this masked
                    # token
                    predictions_ids[i].append(ids_t_i)
                    predictions_detokenized_sents[i].append(pred)
                    
    return predictions_detokenized_sents
        

In [ ]:
gc.collect()

In [ ]:


df_output = {
        'poetry_generated_MHA': poetry,
        'final_predicted_verses' : []
    }

heuristics = []
for i in tqdm(range(0, len(poetry)), position=0, leave=True):
    

    # first get the heuristic masked seqs
    masked_sequences = hueristic(poetry[i])
    heuristics.append(masked_sequences)


    predicted_verses = []
    all_preds = []

    poetry_format = 'M'

    # then for every verse loop and fill the blanks
    for j, each in enumerate(masked_sequences):
        
        filled_verse = predict_seqs_dict(
            final_sequence(
                all_preds, each, j, poetry_format
                ),
            model=model_bert,
            tokenizer=tokenizer_bert, 
            top_k=1,
            order='right-to-left')
        
        all_preds.append(filled_verse)

        
    all_verses = [list(l.values())[-1][0] for l in all_preds]

    for m in range(2, len(all_verses)+1, 2):
        predicted_verses.append(' '.join(all_verses[m-2:m]))
        
    df_output['final_predicted_verses'].append(predicted_verses)

In [ ]:


df_output = {
        'poetry_generated_MHA': poetry,
        'final_predicted_verses' : []
    }

heuristics = []

for i in tqdm(range(0, len(poetry)), position=0, leave=True):
    

    # first get the heuristic masked seqs
    masked_sequences = hueristic(poetry[i])
    heuristics.append(masked_sequences)


    predicted_verses = []
    all_preds, all_preds_tmp = [], []
    poetry_format = 'M'


"""
    # then for every verse loop and fill the blanks
    for j, each in enumerate(masked_sequences):
        
        filled_verse = predict_seqs_dict(
            final_sequence(
                all_preds, each, j, poetry_format
                ),
            model=model_bert,
            tokenizer=tokenizer_bert, 
            top_k=1,
            order='right-to-left')
        
        all_preds.append(filled_verse)
"""
        
    all_verses = [list(l.values())[-1][0] for l in all_preds]

    for m in range(2, len(all_verses)+1, 2):
        predicted_verses.append(' '.join(all_verses[m-2:m]))
        
    df_output['final_predicted_verses'].append(predicted_verses)

## motaradef

In [ ]:
ganjoor_vocab = pd.read_pickle('.../Ganjoor_vocab.pickle')

In [ ]:
for i in range(len(motaradef['synonym'])):

    unw = []

    for j in range(len(motaradef.loc[i, 'synonym'])):

        motaradef['synonym'][i][j] = motaradef['synonym'][i][j].strip()

In [ ]:
motaradef

In [ ]:
motaradef.to_csv('.../motaradef_9May.pickle')

In [ ]:
for i in range(len(motaradef['synonym'])):
    unw = []
    for j in range(len(motaradef.loc[i, 'synonym'])):

        if motaradef.loc[i, 'synonym'][j] not in ganjoor_vocab:
            unw.append(j)
            print(motaradef['synonym'][i][j])

    for m in sorted(unw, reverse=True):
        del motaradef['synonym'][i][m]

In [ ]:
motaradef#.isna().sum()

## Rhyme

In [ ]:
all_poems_beit_address = '.../beits_joined_cleaned.pickle'
rhymes_in_ganjour_address = '.../rhymes_in_ganjour.pickle'
#all_poems = pd.read_pickle(all_poems_beit_address)



In [ ]:
with open(rhymes_in_ganjour_address, 'wb') as f:
    pickle.dump(gh_g, f)

In [ ]:
normalizer = hazm.Normalizer(persian_numbers=False)


def clean(txt):

    txt = normalizer.normalize(txt)

    txt = re.sub(r' \s+', ' ', txt)

    return txt

for i in range(len(all_poems)):
    all_poems[i] = clean(all_poems[i])

In [ ]:

"""
rhymes_in_ganjour = []

for i in all_poems:
    words = i.split()
    first_g = words[-1]
    try: second_g = words[words.index('/')-1]
    except: print(i)
    print(first_g)
    print(second_g)

    #rhymes_in_ganjour.append(first_g)
    #rhymes_in_ganjour.append(second_g)
    """

In [ ]:
#len(rhymes_in_ganjour)

In [ ]:
#len(set(rhymes_in_ganjour))

In [ ]:
rhymes_set = set(rhymes_in_ganjour)

In [ ]:
for n in range(len(rhymes)):

    unw = []

    for i in range(len(rhymes[n])):
        if rhymes[n][i] not in rhymes_set:
            unw.append(i)
            print(rhymes[n][i])

    for j in sorted(unw, reverse=True):
        del rhymes[n][j]

In [ ]:
for i in rhymes:
    for j in i:
        if i =='لعنتی':
            print(i)

In [ ]:
with open('.../rhymes_9May.pickle', 'wb') as f:
    pickle.dump(rhymes, f)

In [ ]:
rhymes

In [ ]:
unw = []
for i in range(len(motaradef['synonym'])):
    if motaradef['synonym'][i] == []:
        print(motaradef.loc[i, 'word'])
        unw.append(i)

motaradef.drop(index = unw, inplace=True)

motaradef.reset_index(inplace=True, drop=True)

In [ ]:
for i in motaradef['synonym']:
    if i == '':
        print('yea')

In [ ]:
for rhyme in rhymes :
    if len(rhyme)<3:
        print('yead')
        print(rhyme)
        

Separating train test for verses


In [ ]:
import glob

!unzip ".../Ganjour/Ganjoor Dataset.zip"

# read all the poetry we have---------------------------------------

all_poems = []

for file_ in glob.glob('*.txt'):
    with open(file_) as f:

        content = f.read()
        content = content.strip().split('\n')
        content = [con for con in content if re.search('\w', con)]
        all_poems = all_poems + content


print('Length of all poems poetries: ', 
      colored(f"{ len(all_poems):,}", 'blue'))


In [ ]:
for i in all_poems:
    if re.search('سلام بر ', i):
        print('yead', i )

In [ ]:
gh_g = []
for m in range(len(poetry)):
    gh_g.append(poetry[m][0].split()[-1])
    gh_g.append(poetry[m][1].split()[-1])


In [ ]:
gh_g = set(gh_g)

In [ ]:
len(gh_g)

In [ ]:
poetry = []
for i in range(2, len(all_poems), 2):
    poetry.append((all_poems[i-2:i]))

In [ ]:
x_train, x_val = train_test_split(all_poems, shuffle = True,
                                                  test_size = 0.1)

In [ ]:
val_path = '.../all_poetry_train_verse_10p_Val.csv'
train_path = '.../all_poetry_val_verse_90p_Train.csv'

In [ ]:

pd.DataFrame(pd.Series(x_train), columns = ['poetry']).to_csv(train_path, index=False)
pd.DataFrame(pd.Series(x_val), columns = ['poetry']).to_csv(val_path, index=False)


In [ ]:
from datasets import load_dataset
dataset_poetry = load_dataset('csv', data_files={'train': train_path,
                                                'test': val_path})

In [ ]:
rhyme_old = pd.read_pickle('.../rhymes_list.pickle')

In [ ]:
def clean_rhyme(t):
    sub_1 = ['أ', ' مسلّم', 'محرّم', 'مقدّم', 'خرّم', 'ّ', 'نوّاب', 'مرکّب', 'محیّا', 'توّاب', 'دوّاب', 'مرکّب', 'محیّا']
    sub_2 = ['ا', ' مسلم', 'محرم', 'مقدم', 'خرم', '', 'نواب', 'مرکب', 'محیا', 'تواب', 'دواب', 'مرکب', 'محیا']

    for i in range(len(sub_1)):
        t = re.sub(sub_1[i], sub_2[i], t)
    return t

In [ ]:
clean_rhymes = []

for rs in rhyme_old:
    cl = []
    for r in rs:
        
        cl.append(clean_rhyme(r))
    clean_rhymes.append(cl)

In [ ]:
clean_rhymes

In [ ]:
unw = []

for i in range(len(rhyme_old)):
    unw = []
    for j in range(len(rhyme_old[i])):
        if rhyme_old[i][j] not in gh_g:
            print(rhyme_old[i][j])
            unw.append(j)

    for m in sorted(unw, reverse=True):
        del rhyme_old[i][m]

In [ ]:
unw = []

for i in range(len(rhyme_old)):
    unw = []
    for j in range(len(rhyme_old[i])):
        if rhyme_old[i][j] not in gh_g:
            print(rhyme_old[i][j])

In [ ]:
with open('.../rhymes_15May.pickle', 'wb') as f:
    pickle.dump(rhymes, f)

## For saving the model

In [ ]:
tokenizer_bert = AutoTokenizer.from_pretrained(model_checkpoint_bert_V2,
                                               use_fast=True)


In [ ]:
tokenizer_bert = AutoTokenizer.from_pretrained(model_checkpoint_bert_V2,
                                               use_fast=True)

In [ ]:
tokenizer_bert.add_tokens(
    ['برآرد', 'برآید', 'وزآن', 'درآمد', 
    'بدانگهی', 'نام‌آو',
    'ناآشنا', 'بدخویی', 'براندیشم']
                      )

model_bert_V2_04_b = AutoModelForMaskedLM.from_pretrained(model_checkpoint_bert_V2)

model_bert_V2_04_b.resize_token_embeddings(len(tokenizer_bert))

In [ ]:
del G_ghafie, G, motaradef, rhymes,

In [ ]:
model_folder_path_bert = r'.../Pretrained Models/BERT_V2_all_poems_mask_0.4/'


model_bert_V2_04_b.load_state_dict(torch.load('.../Pretrained Models/BERT_V2_all_poems_mask_0.4/BERT_V2_all_poems_mask_0.4_onCleanBeits.hpt', 
                                              map_location='cpu'))

#tokenizer_bert = AutoTokenizer.from_pretrained(model_checkpoint_bert, use_fast=True)

In [ ]:
model_bert_V2_04_b.save_pretrained(model_folder_path_bert)